<a href="https://colab.research.google.com/github/MikelKN/new-phd-with-rawat/blob/main/prompt_OPenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install openai
!pip install tqdm
!pip install fire

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import kaggle
from tqdm import tqdm

#displays
from IPython.display import display

# limit to the lenght of token that should be displayed on the daatsets
pd.options.display.max_colwidth = 500

from google.colab import userdata
import os

from openai import OpenAI

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

os.environ["FLOR_OPENAI_KEY"] = userdata.get("FLOR_OPENAI_KEY")

!kaggle datasets download -d elgringofrances/english-hate-speech-superset
!unzip english-hate-speech-superset.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset URL: https://www.kaggle.com/datasets/elgringofrances/english-hate-speech-superset
License(s): MIT
english-hate-speech-superset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  english-hate-speech-superset.zip
replace en_hf_102024.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [21]:
class LLM_sentiment:
    @staticmethod
    def read_and_filter_dataset (filepath, text_column, max_length=200):
        data = pd.read_csv(filepath, low_memory=False)
        # Filter rows based on text length
        data = data[data[text_column].str.len() <= max_length]
        return data

    def sentiment_gpt(self, event, model_name):

      prompt = f"classify this {event} as either positive or negative. omit all explanation and provide only the class"

      client = OpenAI(
          api_key = userdata.get("FLOR_OPENAI_KEY"),  # This is the default and can be omitted
          )

      input = [{"role": "system", "content": "You are a helpful assistant."},
              {"role": "user","content": prompt,}]

      chat_completion = client.chat.completions.create(
          messages=input,
          model=model_name,
      )

      output = chat_completion.choices[0].message.content
      return output

def main(n_test_samples = 5, model_name = 'gpt-4o'):
    llm_sent= LLM_sentiment()

    # Authenticate with Kaggle API
    kaggle.api.authenticate()

    print('\nUploading and reading the datasets starts!')

    # Read the CSV files into DataFrames
    test_df = llm_sent.read_and_filter_dataset('/content/en_hf_102024.csv', 'text')

    #sample just a few use cases
    test_df = test_df.sample(n_test_samples, random_state=123)
    print(f"Sampled {n_test_samples} rows from TEST data")

    #apply sentiment classification
    tqdm.pandas()
    test_df["sentiment"] = test_df["text"].progress_apply(
        lambda x: llm_sent.sentiment_gpt(x, model_name)
    )
    test_df["sentiment"] = test_df["sentiment"].replace(r'\n', ' ', regex=True)

    print('\nThis is the dataset with the sentiment predictions provided by GPT\n')
    display(test_df.head())

if __name__ == "__main__":
    main()



Uploading and reading the datasets starts!
Sampled 5 rows from TEST data


100%|██████████| 5/5 [00:10<00:00,  2.16s/it]


This is the dataset with the sentiment predictions provided by GPT



,text,labels,source,dataset,nb_annotators,tweet_id,post_author_country_location,sentiment
151192,primordial dwarves show us how nature can fail,1.0,synthetic,learning from the worst,3-5,NaN,unknown,Negative
73462,His father Lawrence Kasdan is a Jew as well btw.,0.0,Gab,GHC,3,NaN,unknown,Negative
263123,These people's obsession with race will never cease to amaze,0.0,Reddit,CAD,2+,NaN,unknown,Negative
93478,we must encourage immigrants to be astronauts so they can 🚀🌍👋🏻,1.0,synthetic,hatemoji-build,3,NaN,unknown,Positive
296131,"I don't know if I sympathize with men more than women, but I definitely sympathize with men more than women do.",0.0,Twitter + Reddit + YouTube,measuring-hate-speech,4,NaN,unknown,Negative
